In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Jul 16 00:15:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Vision Module & Embedding Generation

In [ ]:
import torch
#import egg.core as core
from torchvision import models, datasets, transforms
import random
import os.path
from torch.utils.data import Dataset
from typing import Type, Any, Callable, Union, List, Dict, Optional, cast
from collections import OrderedDict 
from torchvision.models.resnet import *
from torchvision.models.resnet import BasicBlock, Bottleneck
from torchvision.models.resnet import model_urls
from torch.hub import load_state_dict_from_url
import csv
import pandas as pd
import time

#import psutil

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load Vision Module & Image Dataset

In [ ]:
# First, the pretrained resnet18 is loaded
rn18 = models.resnet18(pretrained=True)

# Since the output of the resnet 18 model is an almost one-hot encoding, it makes more sense to use an
# intermediate representation. I will use the output of the second to last layer, i.e. the output of the avgpool layer.

# In order to get that desired output, we need to define a new model which is basically a copy of the
# pre trained resnet18 with only the last layer missing. I do this mainly based on https://medium.com/the-owl/extracting-features-from-an-intermediate-layer-of-a-pretrained-model-in-pytorch-easy-way-62631c7fa8f6
class IntResNet(ResNet):
    def __init__(self,output_layer,*args):
        self.output_layer = output_layer
        super().__init__(*args)
        
        self._layers = []
        for l in list(self._modules.keys()):
            self._layers.append(l)
            if l == output_layer:
                break
        self.layers = OrderedDict(zip(self._layers,[getattr(self,l) for l in self._layers]))

    def _forward_impl(self, x):
        for l in self._layers:
            x = self.layers[l](x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

def new_resnet(
    arch: str,
    outlayer: str,
    block: Type[Union[BasicBlock, Bottleneck]],
    layers: List[int],
    pretrained: bool,
    progress: bool,
    **kwargs: Any
) -> IntResNet:

    '''model_urls = {
        'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
        'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
        'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
        'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
        'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
        'resnext50_32x4d': 'https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth',
        'resnext101_32x8d': 'https://download.pytorch.org/models/resnext101_32x8d-8ba56ff5.pth',
        'wide_resnet50_2': 'https://download.pytorch.org/models/wide_resnet50_2-95faca4d.pth',
        'wide_resnet101_2': 'https://download.pytorch.org/models/wide_resnet101_2-32ee1156.pth',
    }'''

    model = IntResNet(outlayer, block, layers, **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
# Second, the vision module is defined as all the layers of the pre trained resnet18 model except the last one
vision_module = new_resnet('resnet18', 'avgpool', BasicBlock, [2, 2, 2, 2],True,True)

# Lastly, the vision module is put into evaluation mode and assigned to the device in use
vision_module.eval()
vision_module.to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:253: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  "Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will "


IntResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)

In [ ]:
imagenet_train_data = datasets.ImageNet(root='/content/drive/My Drive/Thesis_Data/ILSVRC2012', split = 'train', transform = transforms.ToTensor())
#val_data = datasets.ImageNet(root='/content/drive/My Drive/Thesis_Data/ILSVRC2012', split = 'val', transform = transforms.ToTensor())
#imagenet_val_data = datasets.ImageNet(root='/content', split = 'val', transform = transforms.ToTensor())

In [ ]:
train_data_len = int(len(imagenet_train_data)*0.7)
test_data_len = len(imagenet_train_data) - train_data_len
train_data, test_data = torch.utils.data.random_split(imagenet_train_data, [train_data_len, test_data_len], generator=torch.Generator().manual_seed(42))

## Produce & Store Embeddings
Since my computational ressources are limited, the image embeddings that are fed to the sender/receiver are not computed in inference time but ahead of training.

In [ ]:
def produce_game_dataset_txt(img_data, game_data_path, size, test = False):
    start = time.time()
    for i in range(size):
        if i % 1000 == 0:
            end = time.time()
            print(i, ':', end-start)
            start = time.time()
        img_idxs = random.sample(range(0, len(img_data)), k = 3)
        embeddings = []
        labels = []
        for idx in img_idxs:
            img, label = img_data[idx]
            embeddings.append(vision_module(img[None, ...].to(device)))
            labels.append(label)
        target_idx = random.randint(0,2)

        with open(game_data_path, 'a') as game_data:
            row = ''
            for embedding in embeddings:
                embedding_string = str(embedding.tolist()).replace(',', '').replace('[', '').replace(']', '')
                row += embedding_string + ' , '
            row += str(labels[target_idx]) + ' , '
            row += str(target_idx) + '\n'
            game_data.write(row)

In [ ]:
#produce_game_dataset_txt(train_data, '/content/drive/My Drive/Thesis_Data/Game_Data/train.txt', 6633)
#1000 : 1531.020563364029
#2000 : 1532.3743567466736
# 118,661

In [ ]:
#print('Done')

In [ ]:
produce_game_dataset_txt(test_data, '/content/drive/My Drive/Thesis_Data/Game_Data/test.txt', 30000)
# 10,000

0 : 2.1457672119140625e-06
1000 : 2022.2942426204681
2000 : 1898.6195180416107
3000 : 1870.963428735733
4000 : 1839.2745723724365
5000 : 1859.9636926651
6000 : 1833.7077906131744
7000 : 1835.1265635490417
8000 : 1784.2856335639954
9000 : 1802.209911108017
10000 : 1785.7311594486237
11000 : 1748.174738407135
12000 : 1738.773525238037
13000 : 1768.6210618019104
14000 : 1752.8278930187225
15000 : 1710.4765775203705
16000 : 1728.9124355316162
17000 : 1705.9580590724945
18000 : 1701.3090987205505
19000 : 1664.0070970058441
20000 : 1647.4673840999603
21000 : 1612.9175233840942
22000 : 1608.7065370082855
23000 : 1598.1014215946198
24000 : 1564.5836272239685
25000 : 1590.7648718357086
26000 : 1592.0928165912628
27000 : 1537.4044196605682
28000 : 1569.2036199569702
29000 : 1560.9328453540802


In [ ]:
print('Done')

Done


In [ ]:
#val_data = datasets.ImageNet(root='/content/drive/My Drive/Thesis_Data/ILSVRC2012', split = 'val', transform = transforms.ToTensor())

In [ ]:
#produce_game_dataset_txt(val_data, '/content/drive/My Drive/Thesis_Data/Game_Data/val.txt', 20000)
# 10,000

In [ ]:
print('Done')

Done
